In [1]:
%pylab inline
import numpy as np
import scipy.linalg as la
import os,sys,inspect

data_path = '../pulses/output_pulses/'

from quantum_optimal_control.helper_functions.grape_functions import *
from quantum_optimal_control.main_grape.Grape import Grape

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def run_GHZ(qubit_num):


    #Defining time scales
    total_time = qubit_num *2.0
    steps = qubit_num * 10

    # Choose optimizing State transfer or Unitary gate
    state_transfer = True
    
    # Choose whether include intermediate state evolution as part of the graph optimization
    use_inter_vecs = False
    
    #Defining H0
    qubit_state_num = 2
    freq_ge = 0#GHz
    g = 2*np.pi*0.1 #GHz

    Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
    Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
    sigma_z   = np.diag(np.arange(-1,qubit_state_num,2))
    I_q = np.identity(qubit_state_num)
    
    g_op = nn_chain_kron(sigma_z, I_q, qubit_num, qubit_state_num)

    H0 = g * g_op
    

    # Defining dressed info
    is_dressed = False
#     w_c, v_c, dressed_id = get_dressed_info(H0)
#     dressed_info = {'dressed_id':dressed_id, 'eigenvectors':v_c, 'eigenvalues':w_c,'is_dressed':is_dressed}
    
    dressed_info = None


    #Defining Concerned states (starting states)
    g = np.zeros(qubit_state_num**qubit_num,dtype=complex)
    g[0] = 1
    
    psi0=[g]

    all_ones_index = 0
    for ii in range (qubit_num):
        all_ones_index = all_ones_index + qubit_state_num**ii
        
    #Defining states to include in the drawing of occupation
    states_draw_list = [0,all_ones_index]
    states_draw_names = ['00..0','11..1']

    #Defining U (Target)

    if is_dressed:

        gg = get_state_index(0,dressed_id) #[00..0]
        ee = get_state_index(all_ones_index,dressed_id) #[11..1]

        target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
        target_vec = target_vec +  (1)/np.sqrt(2)*v_c[:,gg] + (1)/np.sqrt(2)*v_c[:,ee]

        U = [target_vec]

    else:   
        target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
        target_vec[0]=(1)/np.sqrt(2)
        target_vec[all_ones_index]=1/np.sqrt(2)

        U = [target_vec]
        
    #Defining control Hs

    Hops = []
    Hnames = []
    ops_max_amp = []
    max_amp=2*np.pi*0.5
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'y',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    #Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    
    print Hnames

    #Defining convergence parameters
    max_iterations = 100
    decay = 1000 #max_iterations/2
    convergence = {'rate':0.01, 'update_step':10 ,'max_iterations':max_iterations,\
                   'conv_target':1e-3,'learning_rate_decay':decay,'min_grad':1e-60}

    reg_coeffs = {}
    
    #import h5py
    #with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_hadamard/00002_gpu_spin_chain_hadamard_N9.h5','r') as hf:
    #    u0 = np.array(hf.get('uks'))[-1]
    
    u0=None
    
    

    uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],  

                        show_plots = False, use_gpu = False,sparse_H=False,sparse_U=False,state_transfer = state_transfer, use_inter_vecs = use_inter_vecs,
           unitary_error = 1e-8,  maxA=ops_max_amp,Taylor_terms =[20,0],initial_guess=u0,
                       dressed_info = dressed_info, method = 'ADAM', reg_coeffs=reg_coeffs, file_name='dense_cpu_spin_chain_ghz_N%d' %(qubit_num), 
                        data_path = data_path)

In [3]:
run_GHZ(qubit_num=2)

[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
['xi', 'ix', 'yi', 'iy']
data saved at: ../pulses/output_pulses/00000_dense_cpu_spin_chain_ghz_N2.h5
Using 20 Taylor terms and 0 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :8.61e-01; Runtime: 0.0s; Iterations = 0, grads =   9.570e-01, unitary_metric = 1.00000
Error = :6.36e-02; Runtime: 0.1s; Iterations = 10, grads =   3.440e-01, unitary_metric = 1.00000
Error = :4.75e-02; Runtime: 0.2s; Iterations = 20, grads =   2.395e-01, unitary_metric = 1.00000
Error = :3.07e-03; Runtime: 0.3s; Iterations = 30, grads =   2.190e-02, unitary_metric = 1.00000
Error = :6.57e-04; Runtime: 0.3s; Iterations = 31, grads =   4.951e-03, unitary_metric = 1.00000
data saved at: ../pulses/output_pulses/00000_dense_cpu_spin_chain_ghz_N2.h5
